In [73]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import requests
from dotenv import load_dotenv
import os
import json
import time
import numpy as np

load_dotenv()

True

In [74]:
class TokenTrader:
    def __init__(self, rpc_url, etherscan_api_key, wallet_address, private_key, uniswap_contract_address, token_input_address, token_output_address):
        self.rpc_url = rpc_url
        self.web3 = Web3(Web3.HTTPProvider(rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.etherscan_api_key = etherscan_api_key
        self.wallet_address = wallet_address
        self.private_key = private_key
        # Init Uniswap objects
        self.uniswap_contract_address = self.web3.to_checksum_address(uniswap_contract_address)
        self.uniswap_swap_contract_abi = self.get_contract_abi(self.uniswap_contract_address)
        self.uniswap_swap_contract_object = self.web3.eth.contract(address=self.uniswap_contract_address, abi=self.uniswap_swap_contract_abi)
        # Init Token INPUT objects
        self.token_input_address = self.web3.to_checksum_address(token_input_address)
        self.token_input_abi = self.get_contract_abi(self.token_input_address)
        self.token_input_object = self.web3.eth.contract(address=self.token_input_address, abi=self.token_input_abi)
        self.token_input_symbol = self.token_input_object.functions.symbol().call()
        self.token_input_decimals = self.token_input_object.functions.decimals().call()
        # Init Token OUTPUT objects
        self.token_output_address = self.web3.to_checksum_address(token_output_address)
        self.token_output_abi = self.get_contract_abi(self.token_output_address)
        self.token_output_object = self.web3.eth.contract(address=self.token_output_address, abi=self.token_output_abi)
        self.token_output_symbol = self.token_output_object.functions.symbol().call()
        self.token_output_decimals = self.token_output_object.functions.decimals().call()
        # Init Uniswap object
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=3, web3=self.web3, provider=self.rpc_url)
     
    def get_contract_abi(self, contract_address):
        try:
            url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={self.etherscan_api_key}"
            response = requests.get(url)
            response_json = response.json()
            # Comprueba si la solicitud fue exitosa y si hay un ABI disponible
            if response_json['status'] == '1' and response_json['message'] == 'OK':
                abi = json.loads(response_json['result'])
                return abi
        except Exception as e:
            print(f"Ha ocurrido un error al obtener el ABI: {e}")
            return None
    def get_price(self):
        price = self.uniswap.get_price_input(self.token_input_address, self.token_output_address, 10**self.token_input_decimals)
        return price / 10**self.token_output_decimals
    
    def swap(self, qty):
        print(f"Vendiendo {(qty)} {self.token_input_symbol} a {self.get_price()} {self.token_output_symbol}")
        qty = qty * (10 ** self.token_input_decimals)
        while True:
            try:
                tx = self.uniswap.make_trade(self.token_input_address, self.token_output_address, int(qty))
                print(f"Esperando por confirmación de la transacción {tx.hex()}")
                self.web3.eth.wait_for_transaction_receipt(tx)
                print(f"Transacción confirmada: {tx.hex()}")
                break
            except Exception as e:
                print(f"Transacción fallida, reintentando... Error: {str(e)}\n")
                time.sleep(5)

In [75]:
# Configuración del proveedor
rpc_url = "https://rpc.ankr.com/eth"
etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
wallet_address = os.getenv("WALLET_ADDRESS")
private_key = os.getenv("PRIVATE_KEY")
uniswap_contract_address = "0x1458770554b8918B970444d8b2c02A47F6dF99A7"
token_input_address = "0x26EbB8213fb8D66156F1Af8908d43f7e3e367C1d"
token_output_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
eth_address = "0x0000000000000000000000000000000000000000"
token_trader = TokenTrader(rpc_url,
                           etherscan_api_key,
                           wallet_address, 
                           private_key,
                           uniswap_contract_address,
                           token_input_address,
                           token_output_address)
token_trader.swap(1)

No fee set, assuming 0.3%
No fee set, assuming 0.3%


Vendiendo 1 RETIK a 2.129724 USDT
Vendiendo 1000000000000000000 RETIK a 2.129724 USDT


No fee set, assuming 0.3%


Transacción fallida, reintentando... Error: ('execution reverted', 'no data')




No fee set, assuming 0.3%


...

No fee set, assuming 0.3%


Transacción fallida, reintentando... Error: ('execution reverted', 'no data')




No fee set, assuming 0.3%


Vendiendo 1000000000000000000 RETIK a 2.129724 USDT


KeyboardInterrupt: 